# NY Motor Vehicle Collisions – Exploratory Data Analysis
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95

The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

We look for which borough has had the maximum number of crashes reported since 2012.

We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

Borough	Population
Bronx	1446788
Brooklyn	2648452
Manhattan	1638281
Queens	2330295
Staten Island	487155
We look for the leading cause of crashes

We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident. We look for which combinations of vehicles have the most number of accidents.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [3]:
df = pd.read_pickle(r"Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


### Borough with the second highest number of crashes reported since 2012


In [7]:
#Calculate the number of accidents in each administrative district
borough_crash_counts = df['BOROUGH'].value_counts()
print(borough_crash_counts)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64


### Borough with the minimum number of crashes adjusted for population¶

In [6]:
#Define population estimates for each borough.
population_estimates = {
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
}

#Calculate the crashes per 100,000 people in each borough.
crashes_per_100k = {borough: (crash_count / population_estimates[borough]) * 100000 for borough, crash_count in borough_crash_counts.items()}

print(crashes_per_100k)

{'BROOKLYN': 16652.2179748774, 'QUEENS': 15983.255338916317, 'MANHATTAN': 19121.628096767283, 'BRONX': 14193.164444272415, 'STATEN ISLAND': 11966.827806345003}


### Analyzing the leading cause of crashes

In [8]:
# # Delete the row in which the influencing factor vehicle 1 is "unspecified"
specified_crashes = df[df['CONTRIBUTING FACTOR VEHICLE 1']!= 'Unspecified']

# Find the main causes of crashes and calculate their proportion
leading_cause_count = specified_crashes['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()
total_specified_crashes = specified_crashes['CONTRIBUTING FACTOR VEHICLE 1'].count()
leading_cause_proportion = (leading_cause_count/total_specified_crashes)*100
print(leading_cause_proportion)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction                           30.417970
Failure to Yield Right-of-Way                             9.033469
Following Too Closely                                     8.146617
Backing Unsafely                                          5.688616
Other Vehicular                                           4.752111
Passing or Lane Usage Improper                            4.203050
Turning Improperly                                        3.783314
Passing Too Closely                                       3.778765
Fatigued/Drowsy                                           3.588872
Unsafe Lane Changing                                      3.010322
Traffic Control Disregarded                               2.669121
Driver Inexperience                                       2.370749
Unsafe Speed                                              2.040312
Alcohol Involvement                                       1.641650
Lost Consciousness              

### Top 5 causes of crashes

In [9]:
# Calculate the proportion of total crashes accounted for by the top 5 causes
top_5_causes_proportion = (leading_cause_count.head(5).sum()/total_specified_crashes)*100
print(top_5_causes_proportion)

58.038783742090615


### Accidents involved two or more fatalities

In [10]:
# Treat missing values in "NUMBER OF PERSONS KILLED" as 0 and count accidents with 2 or more fatalities
df['NUMBER OF PERSONS KILLED']=df['NUMBER OF PERSONS KILLED'].fillna(0)
accident_with_2_or_more_fatalities = df[df['NUMBER OF PERSONS KILLED'] >= 2].shape[0]
print(accident_with_2_or_more_fatalities)

88


### At least one person died in every 1000 accidents¶

In [11]:
# Calculate the count of accidents with 1 or more deaths
accident_with_1_or_more_fatalities = df[df['NUMBER OF PERSONS KILLED'] >= 1].shape[0]

# Calculate the total number of accidents
total_accident = df.shape[0]

# Calculate the average number of accidents resulting in at least one death per 1000 accidents
avg_died_per_1000_accidents = (accident_with_1_or_more_fatalities / total_accident) * 1000
print(avg_died_per_1000_accidents)

1.3893258747079764


### Accidents without a Cross Street Name¶

In [12]:
# Calculate the proportion of accidents without a Cross Street Name
accidents_without_Cross_Street_Name = df['CROSS STREET NAME'].isna().sum()
proportion_without_cross_street = (accidents_without_Cross_Street_Name/ total_accident) * 100
print(proportion_without_cross_street)

37.4350983156158


### Combination of vehicles have the most accidents¶

In [14]:
# Calculate the most combination of 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2'
most_vhicles_combination = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size().idxmax()
print(most_vhicles_combination)

('Sedan', 'Sedan')


### Accidents where the contributing factor was Alcohol Involvement

In [15]:
# Filter accidents where the contributing factor was Alcohol Involvement
alcohol_accidents = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Calculate the proportion of these accidents that resulted in fatalities
fatality_in_alcohol_accidents = alcohol_accidents[alcohol_accidents['NUMBER OF PERSONS KILLED']> 0].shape[0]
proportion_alcohol_accidents = (fatality_in_alcohol_accidents /alcohol_accidents.shape[0]) *100
print(proportion_alcohol_accidents)

0.46638345031400075


### Proportion of crashes during the evening rush hour¶

In [24]:
import pandas as pd

# Filter crashes that occur between 4 PM and before 7 PM
evening_rush_hour_df = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

#Calculate the total number of crashes
total_crashes = len(df)
#Calculate the number of crashes during the evening rush hour
evening_rush_hour_crashes = len(evening_rush_hour_df)

# Calculate the proportion of crashes during the evening rush hour
proportion_evening_rush_hour_crashes = (evening_rush_hour_crashes / total_crashes) * 100
print(proportion_evening_rush_hour_crashes)

20.514010935243242


### Proportion resulted in injuries but no fatalities in motorcycles

In [25]:
import pandas as pd

# Correcting for potential missing values interpretation in 'NUMBER OF PERSONS KILLED' and 'NUMBER OF PERSONS INJURED'
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)
df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].fillna(0)

# Filter accidents involving motorcycles in either VEHICLE TYPE CODE 1 or VEHICLE TYPE CODE 2
motorcycle_accidents = df[(df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE', case=False, na=False)) | 
                            (df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE', case=False, na=False))]

# Filter for accidents with injuries but no fatalities
motorcycle_injuries_no_fatalities = motorcycle_accidents[(motorcycle_accidents['NUMBER OF PERSONS INJURED'] > 0) & 
                                                         (motorcycle_accidents['NUMBER OF PERSONS KILLED'] == 0)].shape[0]

# Calculate the proportion of such accidents among motorcycle accidents
proportion_injuries_no_fatalities_motorcycle = (motorcycle_injuries_no_fatalities / motorcycle_accidents.shape[0]) * 100
print(proportion_injuries_no_fatalities_motorcycle)


55.68789838552086


### Crashes involved bicycles as one of the vehicles

In [26]:
import pandas as pd

# Filter accidents involving bicycles in either VEHICLE TYPE CODE 1 or VEHICLE TYPE CODE 2
bicycle_accidents = df[(df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE', case=False, na=False)) | 
                         (df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE', case=False, na=False))].shape[0]
print(bicycle_accidents)

19108
